***Objetivos:*** 
Analizar la eficiencia de inventario de un producto de aguacate real (Avocado Oil).

KPI 1: Riesgo de Desperdicio (Overstock Risk)Definición: ¿Cuánto inventario está por encima de lo que razonablemente se vende, dado el nivel de reorden?Fórmula BI: $\text{Sobre-Inventario} = \text{Stock Quantity} - \text{Reorder Level}$
KPI 2: Eficiencia de Inventario (Inventory Efficiency)Definición: ¿Cuánto vendemos en comparación con lo que tenemos en stock? Si es bajo, significa que la plata está estancada.Fórmula BI: $\text{Eficiencia} = \text{Sales Volume} / \text{Stock Quantity}$

In [ ]:
import os

# --- Ejecuta esta línea en una celda de tu Notebook: ---
print(os.listdir('/kaggle/input/grocery-inventory-and-sales-dataset'))

In [ ]:
import os

# --- Ejecuta esta línea en una celda de tu Notebook: ---
print(os.listdir('/kaggle/input/avocado-prices'))

In [ ]:
import os

# --- Ejecuta esta línea en una celda de tu Notebook: ---
print(os.listdir('/kaggle/input/avocado-prices-20152019'))

In [ ]:
import pandas as pd
import os

# --- RUTAS DE CARGA CORREGIDAS ---

# 1. DATASET DE INVENTARIO Y VENTAS (Tu archivo real)
folder_inventario = '/kaggle/input/grocery-inventory-and-sales-dataset'
nombre_inventario = 'Grocery_Inventory_and_Sales_Dataset.csv' # ¡NOMBRE CONFIRMADO!
file_path_inventario = os.path.join(folder_inventario, nombre_inventario)

# 2. DATASET DE PRECIOS DE AGUACATE (Asumiremos el nombre estándar 'avocado.csv')
folder_precios = '/kaggle/input/avocado-prices' 
nombre_precios = 'avocado.csv'
file_path_precios = os.path.join(folder_precios, nombre_precios)

# 3. DATASET TERCIARIO (Asumiremos un nombre estándar para el ejemplo)
folder_sales = '/kaggle/input/avocado-prices-20152019'
nombre_sales = 'Avocado.csv' # Nombre más probable
file_path_sales = os.path.join(folder_sales, nombre_sales)


# --- FUNCIÓN DE CARGA ---

def cargar_dataframe_seguro(filepath):
    """Carga el archivo CSV y retorna el DataFrame o None si hay error."""
    try:
        df = pd.read_csv(filepath)
        print(f"✅ Cargado: {filepath.split('/')[-1]} con {df.shape[0]} filas.")
        return df
    except FileNotFoundError:
        print(f"🔴 ERROR: Archivo no encontrado. Revise el nombre o la ruta: {filepath.split('/')[-1]}")
        return None
    except Exception as e:
        print(f"❌ Error al leer CSV en {filepath.split('/')[-1]}: {e}")
        return None

# --- CARGA FINAL ---

df_inventario = cargar_dataframe_seguro(file_path_inventario)
df_precios = cargar_dataframe_seguro(file_path_precios)
df_sales = cargar_dataframe_seguro(file_path_sales)

print("\n--- ¡Listos para el Análisis! ---")

In [ ]:
print("Columnas de df_inventario:")
print(df_inventario.columns)
print("\nPrimeras 5 filas de df_inventario:")
print(df_inventario.head())

In [ ]:
print("Columnas de df_inventario:")
print(df_precios.columns)
print("\nPrimeras 5 filas de df_inventario:")
print(df_precios.head())

In [ ]:
print("Columnas de df_inventario:")
print(df_sales.columns)
print("\nPrimeras 5 filas de df_inventario:")
print(df_sales.head())

In [ ]:
# 1. Buscamos si "Avocado" está en la columna Product_Name
df_aguacate_encontrado = df_inventario[df_inventario['Product_Name'].str.contains('Avocado|Aguacate', case=False, na=False)]

if not df_aguacate_encontrado.empty:
    print(f"✅ Aguacate encontrado! {df_aguacate_encontrado.shape[0]} registros.")
    df_inventario_producto = df_aguacate_encontrado
else:
    # 2. Si no se encuentra, usamos toda la categoría 'Fruits & Vegetables' como proxy
    print("❌ Aguacate específico no encontrado. Usando 'Fruits & Vegetables' como proxy de producto perecedero.")
    df_inventario_producto = df_inventario[df_inventario['Catagory'] == 'Fruits & Vegetables']
    
print(f"Usaremos {df_inventario_producto.shape[0]} registros para el análisis de inventario.")

In [ ]:
# --- Creación del KPI de Eficiencia ---

# NOTA: Agregamos un chequeo para evitar la división por cero si Stock_Quantity es 0
df_inventario_producto['Eficiencia_Inventario'] = (
    df_inventario_producto['Sales_Volume'] / 
    df_inventario_producto['Stock_Quantity'].replace(0, 1) # Si es 0, lo ponemos a 1 para evitar error.
)

# Mostramos los nuevos KPIs creados para verificar
print("\nKPIs Creados (5 primeras filas):")
print(df_inventario_producto[['Product_Name', 'Stock_Quantity', 'Sales_Volume', 'Reorder_Level', 'Sobre_Inventario', 'Eficiencia_Inventario']].head())

In [ ]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go # Necesario para las líneas de forma

# --- NOTA: Asegúrate de que df_analisis ya está definido y contiene los KPIs ---

# 1. Gráfico de Dispersión (Scatter Plot) sin los argumentos problemáticos
fig_riesgo_inventario = px.scatter(
    df_analisis,
    x='Eficiencia_Inventario',
    y='Sobre_Inventario', 
    color='Product_Name',
    size='Stock_Quantity', 
    hover_name='Product_Name',
    title='Análisis de Riesgo Logístico: Avocado Oil (Data Actual 2024)',
    labels={
        'Eficiencia_Inventario': 'Eficiencia (Ventas / Stock)',
        'Sobre_Inventario': 'Riesgo de Sobre-Stock'
    },
    template='plotly_dark' 
)

# 2. AÑADIR LÍNEAS DE REFERENCIA DE RIESGO (CORRECCIÓN)
# Establecemos las reglas de negocio (Umbrales de Riesgo):
UMBRAL_SOBRE_STOCK = 10  # Si Sobre_Inventario es > 10, hay riesgo alto
UMBRAL_BAJA_EFICIENCIA = 0.5 # Si Eficiencia es < 0.5, hay riesgo de dinero estancado

# Línea Horizontal (Riesgo de Sobre-Stock):
fig_riesgo_inventario.add_shape(
    type="line",
    x0=df_analisis['Eficiencia_Inventario'].min() * 0.9, # Inicio del eje X
    y0=UMBRAL_SOBRE_STOCK, 
    x1=df_analisis['Eficiencia_Inventario'].max() * 1.1, # Fin del eje X
    y1=UMBRAL_SOBRE_STOCK,
    line=dict(color="Red", width=2, dash="dash"),
    name='Riesgo Alto de Almacenamiento'
)

# Línea Vertical (Riesgo de Baja Eficiencia):
fig_riesgo_inventario.add_shape(
    type="line",
    x0=UMBRAL_BAJA_EFICIENCIA,
    y0=df_analisis['Sobre_Inventario'].min() * 0.9, # Inicio del eje Y
    x1=UMBRAL_BAJA_EFICIENCIA,
    y1=df_analisis['Sobre_Inventario'].max() * 1.1, # Fin del eje Y
    line=dict(color="Yellow", width=2, dash="dash"),
    name='Baja Rotación/Ineficiencia'
)

# 3. Configuración Final
fig_riesgo_inventario.update_layout(
    xaxis_title='Eficiencia de Rotación',
    yaxis_title='Riesgo de Sobre-Stock (Capital Inmovilizado)',
    showlegend=True
)

fig_riesgo_inventario.show()

In [ ]:
fig_riesgo_inventario.show()